<a href="https://colab.research.google.com/github/alfrepereyra/laboratorio/blob/main/spotify_laboratorio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


 **TODAS LAS IMPORTACIONES NECESARIAS**



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
from sklearn.ensemble import VotingClassifier


# 1-Elegir las características óptimas para el entrenamiento de los modelos


necesito normalizar los datos de este csv y crear un nuevo dataframe, lo primero quitar todos los datos nulos o Nan y despues convertir todos los datos numericos a float entre 0 y 1 para poder trabajar mejor:



In [ ]:

archivo_csv = '/content/drive/MyDrive/UPSO/laboratorio_Spotify/Canciones_Spotify.csv'

#lo convertimos a df
df = pd.read_csv(archivo_csv)

#Elimino datos nulos
df = df.dropna()

#Normalizo datos numéricos entre 0 y 1
scaler = MinMaxScaler()
columnas_numericas = df.select_dtypes(include=['float64']).columns
df[columnas_numericas] = scaler.fit_transform(df[columnas_numericas])

#creo el nuevo df normalizado
df_nuevo = pd.DataFrame(df)
print(df_nuevo)


      Unnamed: 0  acousticness  danceability  duration_ms    energy  \
0              0      0.010248      0.824826       204600  0.426363   
1              1      0.199998      0.720418       326933  0.350081   
2              2      0.034570      0.830626       185707  0.403987   
3              3      0.607034      0.431555       199413  0.328723   
4              4      0.180902      0.645012       392893  0.555533   
...          ...           ...           ...          ...       ...   
2012        2012      0.001062      0.535963       274404  0.932872   
2013        2013      0.088138      0.895592       182182  0.892189   
2014        2014      0.008610      0.597448       207200  0.935924   
2015        2015      0.001645      0.504640       185600  0.993897   
2016        2016      0.002821      0.375870       204520  0.915582   

      instrumentalness  key  liveness  loudness  mode  speechiness     tempo  \
0             0.022439    2  0.153862  0.741141     1     0.514441 

#2- Separar en datos de entrenamiento y datos de prueba
**Realizar varios modelos de Machine Learning:**
- KNN (K-Nearest Neighbors)
- SVM (Support Vector Machines)
- Árbol de decisión
- Bayes (Naive Bayes)
- Otro modelo que crean conveniente
 divido el conjunto de datos normalizado en conjuntos de entrenamiento y prueba. quitando las columnas de artist y song title que puede dar problemas y aplico 4 modelos de machine learning

In [ ]:

# Elimina la columna 'song_title' y 'artist'que no son numericos
X = df_nuevo.drop(['target', 'song_title',"artist"], axis=1)


y = df_nuevo['target']

# Divide el conjunto de datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#SVM
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)
# Hacer predicciones en el conjunto de prueba
svm_predictions = svm_model.predict(X_test)
# Evaluar el rendimiento
svm_accuracy = accuracy_score(y_test, svm_predictions)
print(f'SVM Accuracy: {svm_accuracy}')

#KNN
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)
# Hacer predicciones en el conjunto de prueba
knn_predictions = knn_model.predict(X_test)
# Evaluar el rendimiento
knn_accuracy = accuracy_score(y_test, knn_predictions)
print(f'KNN Accuracy: {knn_accuracy}')

#Árbol de decisión
tree_model = DecisionTreeClassifier(max_depth=3)
tree_model.fit(X_train, y_train)
# Hacer predicciones en el conjunto de prueba
tree_predictions = tree_model.predict(X_test)
# Evaluar el rendimiento
tree_accuracy = accuracy_score(y_test, tree_predictions)
print(f'Decision Tree Accuracy: {tree_accuracy}')


#Naive Bayes
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)
# Hacer predicciones en el conjunto de prueba
nb_predictions = nb_model.predict(X_test)
# Evaluar el rendimiento
nb_accuracy = accuracy_score(y_test, nb_predictions)
print(f'Gaussian Naive Bayes Accuracy: {nb_accuracy}')


SVM Accuracy: 1.0
KNN Accuracy: 0.8910891089108911
Decision Tree Accuracy: 1.0
Gaussian Naive Bayes Accuracy: 0.9900990099009901


#3-Realizar algún tipo de validación:
- Validación Simple
- Validación Cruzada k-fold

In [ ]:


# X = df_nuevo.drop(['target', 'song_title', 'artist'], axis=1)


# y = df_nuevo['target']

# # Divide el conjunto de datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelos
models = {
    'SVM': SVC(kernel='linear'),
    'KNN': KNeighborsClassifier(n_neighbors=5),
    'Decision Tree': DecisionTreeClassifier(max_depth=3),
    'Gaussian Naive Bayes': GaussianNB()
}

# Entrenamiento y evaluación
for model_name, model in models.items():
    # Entrenamiento
    model.fit(X_train, y_train)

    # Predicciones
    predictions = model.predict(X_test)

    # Evaluación
    accuracy = accuracy_score(y_test, predictions)
    print(f'{model_name} Accuracy: {accuracy}')

    # Validación Cruzada k-fold
    cv_scores = cross_val_score(model, X, y, cv=5)
    print(f'{model_name} Cross-Validation Scores: {cv_scores}')
    print(f'{model_name} Mean Cross-Validation Score: {cv_scores.mean()}\n')

SVM Accuracy: 1.0
SVM Cross-Validation Scores: [0.75990099 1.         1.         1.         0.74937965]
SVM Mean Cross-Validation Score: 0.9018561285408937

KNN Accuracy: 0.8910891089108911
KNN Cross-Validation Scores: [0.78960396 0.89356436 0.83126551 0.84615385 0.75186104]
KNN Mean Cross-Validation Score: 0.8224897427708033

Decision Tree Accuracy: 1.0
Decision Tree Cross-Validation Scores: [0.75247525 1.         1.         1.         0.74689826]
Decision Tree Mean Cross-Validation Score: 0.8998747021104094

Gaussian Naive Bayes Accuracy: 0.9900990099009901
Gaussian Naive Bayes Cross-Validation Scores: [0.78960396 1.         0.93548387 1.         0.77171216]
Gaussian Naive Bayes Mean Cross-Validation Score: 0.8993599980345429



#4-Evaluación y análisis del rendimiento de cada modelo:
- Matriz de confusión
- Precisión, recall y F1-score

In [ ]:


# Función para mostrar la matriz de confusión en un mapa de calor
def mostrar_matriz_confusion(conf_matrix, title):
    plt.figure(figsize=(6, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title(title)
    plt.xlabel('Predicho')
    plt.ylabel('Real')
    plt.show()

# Función para mostrar las métricas de clasificación
def mostrar_metricas_clasificacion(precision, recall, f1, title):
    labels = ['Precisión', 'Recall', 'F1-Score']
    values = [precision, recall, f1]

    plt.figure(figsize=(8, 4))
    bars = plt.bar(labels, values, color=['green', 'orange', 'blue'])
    plt.title(title)
    plt.ylabel('Puntuación')

    # Mostrar los valores de las barras
    for bar, value in zip(bars, values):
        plt.text(bar.get_x() + bar.get_width() / 2 - 0.1, bar.get_height() + 0.02, f'{value:.2f}', fontsize=9)

    plt.show()

# Mostrar la matriz de confusión para SVM
mostrar_matriz_confusion(svm_conf_matrix, 'Matriz de Confusión SVM')
mostrar_metricas_clasificacion(svm_precision, svm_recall, svm_f1_score, 'Métricas de Clasificación SVM')

# Mostrar la matriz de confusión para KNN
mostrar_matriz_confusion(knn_conf_matrix, 'Matriz de Confusión KNN')
mostrar_metricas_clasificacion(knn_precision, knn_recall, knn_f1_score, 'Métricas de Clasificación KNN')

# Mostrar la matriz de confusión para Árbol de decisión
mostrar_matriz_confusion(tree_conf_matrix, 'Matriz de Confusión Árbol de decisión')
mostrar_metricas_clasificacion(tree_precision, tree_recall, tree_f1_score, 'Métricas de Clasificación Árbol de decisión')

# Mostrar la matriz de confusión para Naive Bayes
mostrar_matriz_confusion(nb_conf_matrix, 'Matriz de Confusión Naive Bayes')
mostrar_metricas_clasificacion(nb_precision, nb_recall, nb_f1_score, 'Métricas de Clasificación Naive Bayes')

#5- Ajustar para cada uno de los modelos los hiper parámetros:
- Grid Search
- Random Search


##**GRID SEARCH**
Este bloque de codigo suele tardar mucho en ejecutarse pero funciona (en vsc me tardo 3minutos) ya que que se están ajustando 18 combinaciones de hiperparámetros en un conjunto de entrenamiento dividido en 5 pliegues (folds), lo que resulta en un total de 90 ajustes (fits). Esto se debe al uso de la validación cruzada en el proceso de Grid Search. utilize 'n_jobs' en el objeto GridSearchCV' para utiliza todos los núcleos disponibles ede la pc

In [ ]:


X = df_nuevo.drop(['target', 'song_title', 'artist'], axis=1)


y = df_nuevo['target']

# Divide el conjunto de datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelo SVM
svc = SVC()
param_grid_svc = {
    'kernel': ['linear', 'rbf'],
    'C': [0.1, 1, 10],
    'gamma': [0.1, 1, 10]
}


grid_search_svc = GridSearchCV(svc, param_grid_svc, cv=5, scoring='accuracy' , n_jobs=-1 )
grid_search_svc.fit(X_train, y_train)

# Modelo KNN
knn = KNeighborsClassifier()
param_grid_knn = {
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}
grid_search_knn = GridSearchCV(knn, param_grid_knn, cv=5, scoring='accuracy', n_jobs=-1 )
grid_search_knn.fit(X_train, y_train)

# Modelo Árbol de decisión
tree = DecisionTreeClassifier()
param_grid_tree = {
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid_search_tree = GridSearchCV(tree, param_grid_tree, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_tree.fit(X_train, y_train)

# Modelo Naive Bayes (Gaussian)
nb = GaussianNB()
param_grid_nb = {}
grid_search_nb = GridSearchCV(nb, param_grid_nb, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_nb.fit(X_train, y_train)

# Imprimir resultados para SVM
print("Mejores parámetros para SVM:", grid_search_svc.best_params_)
print("Mejor score para SVM:", grid_search_svc.best_score_)

# Imprimir resultados para KNN
print("Mejores parámetros para KNN:", grid_search_knn.best_params_)
print("Mejor score para KNN:", grid_search_knn.best_score_)

# Imprimir resultados para Árbol de decisión
print("Mejores parámetros para Árbol de decisión:", grid_search_tree.best_params_)
print("Mejor score para Árbol de decisión:", grid_search_tree.best_score_)

# Imprimir resultados para Naive Bayes
print("Mejores parámetros para Naive Bayes:", grid_search_nb.best_params_)
print("Mejor score para Naive Bayes :", grid_search_nb.best_score_)

Mejores parámetros para SVM: {'C': 0.1, 'gamma': 0.1, 'kernel': 'linear'}
Mejor score para SVM: 0.9987616099071207
Mejores parámetros para KNN: {'n_neighbors': 3, 'p': 1, 'weights': 'distance'}
Mejor score para KNN: 0.902638309328308
Mejores parámetros para Árbol de decisión: {'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2}
Mejor score para Árbol de decisión: 0.9987577639751553
Mejores parámetros para Naive Bayes: {}
Mejor score para Naive Bayes : 0.983870161336846


##**RANDOM SEARCH**

random search realiza búsquedas aleatorias en el espacio de parámetros y podría ser más eficiente para un gran número de combinaciones. tambien puede tardar un poco


In [ ]:

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from scipy.stats import uniform, randint

# Definir modelos y distribuciones de parámetros para Random Search
models_distributions = {
    'SVM': (SVC(), {'kernel': ['linear', 'rbf', 'poly'], 'C': uniform(0.1, 10), 'gamma': ['scale', 'auto']}),
    'KNN': (KNeighborsClassifier(), {'n_neighbors': randint(1, 20), 'weights': ['uniform', 'distance'], 'p': [1, 2]}),
    'Árbol de decisión': (DecisionTreeClassifier(), {'max_depth': randint(1, 20), 'min_samples_split': randint(2, 20), 'min_samples_leaf': randint(1, 10)}),
    'Naive Bayes (Gaussian)': (GaussianNB(), {})
}

# Realizar Random Search para cada modelo con paralelización
for model_name, (model, dist) in models_distributions.items():
    random_search = RandomizedSearchCV(model, dist, n_iter=10, scoring='f1_macro', cv=5, n_jobs=-1, random_state=42)
    random_search.fit(X_train, y_train)

    # Imprimir resultados para cada modelo
    print(f"\nMejores parámetros para {model_name}:", random_search.best_params_)
    print(f"Mejor score para {model_name}:", random_search.best_score_)

#6-Realizar un ensamble de los modelos:
-Votación Mayoritaria

In [5]:


# modelos entrenados
svm_modelo = SVC(C=0.1, kernel='linear')
knn_modelo = KNeighborsClassifier(n_neighbors=5, weights='uniform', p=2)
tree_modelo = DecisionTreeClassifier(max_depth=5, min_samples_split=2, min_samples_leaf=1)
nb_modelo = GaussianNB()

# creo el ensamble con votación mayoritaria
ensamble_modelo = VotingClassifier(estimators=[
    ('svm', svm_modelo),
    ('knn', knn_modelo),
    ('tree', tree_modelo),
    ('nb', nb_modelo)
], voting='hard')

# entrenar el ensamble en el conjunto de entrenamiento
ensamble_modelo.fit(X_train, y_train)

# predecir en el conjunto de prueba
ensamble_prediccion = ensemble_model.predict(X_test)

# evaluo la precisión del ensamble
ensamble_puntaje = accuracy_score(y_test, ensamble_prediccion)
print(f'Precisión del Ensamble (Votación Mayoritaria): {ensamble_puntaje}')

NameError: ignored

#7-Evaluación y análisis del rendimiento:
- Matriz de confusión
- Precisión, recall y F1-score


In [3]:

# calculo la matriz de confusión
conf_matrix = confusion_matrix(y_test, ensamble_prediccion)
print("Matriz de Confusión:")
print(conf_matrix)

# calculo precisión, recall y F1-score
precision = precision_score(y_test, ensamble_prediccion)
recall = recall_score(y_test, ensamble_prediccion)
f1 = f1_score(y_test, ensamble_prediccion)



# visualizo la matriz de confusión como un heatmap
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['No Gustó (0)', 'Gustó (1)'], yticklabels=['No Gustó (0)', 'Gustó (1)'])
plt.xlabel('Predicciones')
plt.ylabel('Valores Reales')
plt.title('Matriz de Confusión - Ensamble con Votación Mayoritaria')
plt.show()

print(f'Precisión: {precision}')
print(f'Recall: {recall}')
print(f'F1-score: {f1}')

NameError: ignored